In [34]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

#Definimos una funcion para tratar a las imagenes (en vez de una por una como hice en el commit pasado):
def load_image(path, size=(64,64)):
    img = Image.open(path).resize(size)
    arr = np.array(img) / 255.0     # normalizar
    return arr.flatten().reshape(-1,1)  # vector columna

#Tomamos nuestras imagenes y les aplicamos la funcion:
x1 = load_image("Img/Dog1.jpg")
x2 = load_image("Img/Dog2.jpg")
x3 = load_image("Img/Dog3.jpg")
x4 = load_image("Img/Cat1.jpeg")
x5 = load_image("Img/Cat2.jpg")
x6 = load_image("Img/Cat3.jpg")

# Nuestras X son:
X = np.concatenate([x1, x2, x3, x4, x5, x6], axis=1)
print("Dimensiones de X:", X.shape) 

# Etiquetas con perros=0 y gatos=1:
Y = np.array([[0, 0, 0, 1, 1, 1]])  
print("Dimensiones de Y:", Y.shape) 

#Ahora si procedemos a inicializar los datos, usamos aleatoriedad para ayudar al aprendizaje de nuestro modelo:

np.random.seed(1)                       #Con esto fijamos un numero aleatorio para evitar que cambie cada que se corra el codigo
n_x = X.shape[0]                        #Para extraer los pixeles que tiene cada imagen
w = np.random.randn(n_x, 1) * 0.01      #Le doy numeros chicos para evitar sobrecarga en el calculo
b = 0
a= 0.1                                  # (es la tasa de aprendizaje)
iteraciones = 1000                                # numero de iteraciones

#Definimos una funcion para nuestra funcion sigmoide:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

#Definimos otra funcion para calcular el costo, dw y db que nos ayudaran a hacer las backward y forward propagations
def propagaciones(w, b, X, Y):
    m = X.shape[1]
    # Forward
    Z = np.dot(w.T, X) + b
    A = sigmoide(Z)                      #Este es nuestro vector de predicciones "Y con gorrito"
    costo = -(1/m) * np.sum(Y*np.log(A+1e-8) + (1-Y)*np.log(1-A+1e-8))   #el 1e-8 es para evitar errores numericos en caso de tener 0 o 1
    # Backward
    dZ = A - Y
    dw = (1/m) * np.dot(X, dZ.T)
    db = (1/m) * np.sum(dZ)
    return dw, db, costo

def mejoramiento(w, b, X, Y, iteraciones, a):
    for i in range(iteraciones):
        dw, db, costo = propagaciones(w, b, X, Y)
        # Gradient descent
        w -= a * dw
        b -= a * db
        #Para ir viendo si nuestro si nuestro costo va bajando con el entrenamiento aplicamos:
        if i % 200 == 0:
            print(f"En la iteracion numero {i}, Tenemos un costo de: {costo:.4f}")
    return w, b

#Ahora si viene el entrenamiento:
w, b = mejoramiento(w, b, X, Y, iteraciones, a)

#Y finalmente:
def predicciones(X, w, b):
    A = sigmoide(np.dot(w.T, X) + b)
    return (A > 0.5).astype(int)        #Aqui aplicamos el umbral de desicion y convertimos el True/False en 0/1

#Vemos que tanto coincide:
predicc = predicciones(X, w, b)
print("Predicciones:", predicc)
print("Etiquetas reales:", Y)

#Ahora si probamos el algoritmo con una imagen nueva, recordando que perros=0 y gatos=1:
nueva = load_image("Img/gatonuevo3.jpg")
pred_nueva = predicciones(nueva, w, b)
print("Predicción de nueva imagen:", pred_nueva)


Dimensiones de X: (12288, 6)
Dimensiones de Y: (1, 6)
En la iteracion numero 0, Tenemos un costo de: 0.6710
En la iteracion numero 200, Tenemos un costo de: -0.0000
En la iteracion numero 400, Tenemos un costo de: -0.0000
En la iteracion numero 600, Tenemos un costo de: -0.0000
En la iteracion numero 800, Tenemos un costo de: -0.0000
Predicciones: [[0 0 0 1 1 1]]
Etiquetas reales: [[0 0 0 1 1 1]]
Predicción de nueva imagen: [[1]]
